# 데이터 정보 크롤링 (pdf 링크 / 날짜 / 제목 / 증권사이름 / pdf 파일이름 / txt 파일이름)

In [8]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def crawl_info():
    total_info = []
    page = 1  # 페이지 초기값 설정
    while True:
        URL = f'https://finance.naver.com/research/debenture_list.naver?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2014-01-01&writeToDate=2023-12-31&x=46&y=21&page={page}'
        response = requests.get(URL)
        soup = BeautifulSoup(response.text, 'html.parser')
        table_tags = soup.select('tr')
        info = []
        for table_tag in table_tags:
            try:
                td_tags = table_tag.select('td')
                if len(td_tags) >= 2:
                    title_tag = td_tags[0].select_one('a')
                    pdf_tags = table_tag.select('td.file > a')
                    date_tags = table_tag.select('td.date')
                    broker_name = td_tags[1].text.strip()  # 증권사 이름 가져오기
                    if pdf_tags and date_tags:
                        pdf = pdf_tags[0].attrs['href']
                        date = date_tags[0].text
                        title = title_tag.text
                        pattern = '\\d+\\.pdf'
                        file_name = re.findall(pattern, pdf)
                        file_name = date + '_' + file_name[0]
                        info.append((pdf, date, title, broker_name, file_name))
            except Exception as e:
                print(f'Error: {e}')
        total_info.extend(info)
        
        # "맨뒤" 버튼이 없을 경우 탐색을 멈춤
        last_button = soup.select_one('td.pgRR > a')
        if not last_button:
            break
        
        page += 1  # 다음 페이지로 이동

    # 데이터프레임으로 변환
    df = pd.DataFrame(total_info, columns=["pdf_link", "date", "title", "broker_name", "file_name"])
    df['content_file'] = df['file_name'].apply(lambda x: x.replace('.pdf', '.txt'))

    # 데이터프레임을 CSV 파일로 저장
    df.to_csv("pdf_link_crawl_add_txt.csv", sep='\t', index=False)
    return 'pdf_link_crawl_add_txt.csv'

In [9]:
crawl_info()

'pdf_link_crawl_add_txt.csv'

# pdf download

In [7]:
import re
import urllib.request
import pandas as pd
import os
import traceback

def pdf_downloader(csv_file_name):

    # CSV 파일을 불러와서 데이터프레임으로 변환
    df = pd.read_csv(csv_file_name, sep='\t')

    # 데이터프레임을 리스트 안의 튜플 데이터 형태로 변환
    total_info = [tuple(row) for row in df.values]

    dir = './reportpdf/'
    if not os.path.exists(dir):
        os.makedirs(dir)

    for i in range(len(total_info)):
        if i==0:
            print('pdf 다운로드 시작')
        try:
            pattern='\\d+\\.pdf'
            file_name = re.findall(pattern, total_info[i][0])
            file_name = total_info[i][1]+'_'+file_name[0]
            urllib.request.urlretrieve(total_info[i][0], dir+file_name)
        except:
            print(f"error : {total_info[i][0]}")
            print(traceback.format_exc())
            i += 1 # 에러가 발생한 경우, 해당 항목부터 다시 크롤링할 수 있도록 i를 1 증가시킴
        if i!=0 and i%100==0:
            print(f'pdf 다운로드 진행률 : {i}/{len(total_info)}') # 100개 다운로드마다 알림

In [4]:
pdf_downloader('pdf_link_crawl_add_txt.csv')

pdf 다운로드 시작
pdf 다운로드 진행률 : 100/4898
pdf 다운로드 진행률 : 200/4898
pdf 다운로드 진행률 : 300/4898
pdf 다운로드 진행률 : 400/4898
pdf 다운로드 진행률 : 500/4898
pdf 다운로드 진행률 : 600/4898
pdf 다운로드 진행률 : 700/4898
pdf 다운로드 진행률 : 800/4898
pdf 다운로드 진행률 : 900/4898
error : https://ssl.pstatic.net/imgstock/upload/research/debenture/1688340332432.pdf
Traceback (most recent call last):
  File "C:\Users\SDA05\AppData\Local\Temp\ipykernel_10676\3619605239.py", line 26, in pdf_downloader
    urllib.request.urlretrieve(total_info[i][0], dir+file_name)
  File "c:\Users\SDA05\anaconda3\envs\mecab\lib\urllib\request.py", line 278, in urlretrieve
    raise ContentTooShortError(
urllib.error.ContentTooShortError: <urlopen error retrieval incomplete: got only 265686 out of 466542 bytes>

pdf 다운로드 진행률 : 1000/4898
pdf 다운로드 진행률 : 1100/4898
pdf 다운로드 진행률 : 1200/4898
pdf 다운로드 진행률 : 1300/4898
pdf 다운로드 진행률 : 1400/4898
error : https://ssl.pstatic.net/imgstock/upload/research/debenture/1681692957681.pdf
Traceback (most recent call last):
  File 

에러 발생한 pdf의 경우, 따로 다운로드하여 폴더에 저장함